In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt
import re

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [12]:
bets = pd.read_csv('In_Season/Data/todays_bets.csv', index_col = 0)
bets

,Away,Home,is_B2B_First_Away,is_B2B_First_Home,is_B2B_Second_Away,is_B2B_Second_Home,Away_Prob_Naive,Home_Prob_Naive,Home_Prob_Adjusted,Away_Prob_Adjusted,...,Home_Prob_Odds,Away_Prob_Odds,Home_Prob_Diff,Away_Prob_Diff,Home_Bet_Size,Away_Bet_Size,Home_Bet,Away_Bet,Potential_Payoff,Date
0,San Antonio Spurs,Cleveland Cavaliers,0,0,0,0,0.456188,0.571590,0.712207,0.287793,...,0.705882,0.335570,0.006325,-0.047778,0.002151,0,218.916178,0.0,91.215074,2022-02-09
1,Chicago Bulls,Charlotte Hornets,0,0,0,0,0.360424,0.396679,0.624630,0.375370,...,0.480769,0.557522,0.143861,-0.182152,0.027707,0,2820.442145,0.0,3046.077516,2022-02-09
2,Toronto Raptors,Oklahoma City Thunder,1,0,0,0,0.507172,0.212577,0.241065,0.758935,...,0.206186,0.833333,0.034879,-0.074398,0.004394,0,447.280981,0.0,1722.031776,2022-02-09
3,Los Angeles Lakers,Portland Trailblazers,0,0,1,1,0.420005,0.242449,0.355536,0.644464,...,0.232558,0.807692,0.122978,-0.163228,0.016024,0,1631.233399,0.0,5383.070217,2022-02-09
4,Minnesota Timberwolves,Sacramento Kings,0,0,1,1,0.499497,0.351475,0.408243,0.591757,...,0.227273,0.811321,0.180970,-0.219564,0.023420,0,2384.048566,0.0,8105.765124,2022-02-09
5,Golden State Warriors,Utah Jazz,1,0,0,0,0.611055,0.582495,0.545609,0.454391,...,0.553571,0.485437,-0.007962,-0.031046,0.000000,0,0.000000,0.0,0.000000,2022-02-09


In [17]:
results = pd.read_csv('In_Season/Data/results_tracker.csv', index_col = 0)
results = results.iloc[:, [1:]]
results

,Unnamed: 0.1,Away,Home,is_B2B_First_Away,is_B2B_First_Home,is_B2B_Second_Away,is_B2B_Second_Home,Away_Prob_Naive,Home_Prob_Naive,Home_Prob_Adjusted,...,Home_Prob_Diff,Away_Prob_Diff,Home_Bet_Size,Away_Bet_Size,Home_Bet,Away_Bet,Potential_Payoff,Date,Won_Bet,Capital
0,0.0,Philadelphia 76ers,Chicago Bulls,0,1,0,0,0.528106,0.444929,0.484108,...,0.003339,-0.047427,0.000643,0.000000,64.302009,0.000000,69.446169,2022-02-07,0,99935.697991
1,1.0,Brooklyn Nets,Denver Nuggets,0,0,0,0,0.374566,0.533183,0.760983,...,0.007897,-0.050838,0.003198,0.000000,319.820050,0.000000,104.859033,2022-02-07,1,100040.557024
2,2.0,Detroit Pistons,Minnesota Timberwolves,0,0,0,0,0.239436,0.432112,0.820522,...,-0.064536,0.025632,0.000000,0.003029,0.000000,302.922247,1666.072357,2022-02-07,0,99737.634777
3,3.0,Indiana Pacers,Cleveland Cavaliers,0,0,0,0,0.281105,0.497420,0.831493,...,0.154074,-0.195130,0.047763,0.000000,4776.293883,0.000000,2274.425659,2022-02-07,1,102012.060436
4,4.0,Atlanta Hawks,Dallas Mavericks,0,0,0,0,0.471007,0.476521,0.586414,...,0.023095,-0.069506,0.005289,0.000000,528.881340,0.000000,409.985535,2022-02-07,1,102422.045971
5,5.0,Boston Celtics,Orlando Magic,0,0,0,1,0.551662,0.250972,0.235429,...,0.018038,-0.063015,0.002305,0.000000,230.482670,0.000000,829.737612,2022-02-07,0,102191.563301
6,6.0,New Orleans Pelicans,Houston Rockets,0,0,0,0,0.348352,0.203080,0.374466,...,-0.010150,-0.035483,0.000000,0.000000,0.000000,0.000000,0.000000,2022-02-07,-1,102191.563301
7,7.0,Milwaukee Bucks,Los Angeles Clippers,0,0,1,0,0.604617,0.399706,0.382212,...,-0.062232,0.017788,0.000000,0.004447,0.000000,444.698863,296.465909,2022-02-07,1,102488.029210
8,NaN,Phoenix Suns,Philadelphia 76ers,0,0,1,0,0.644449,0.497934,0.445611,...,-0.087100,0.039826,0.000000,0.008204,0.000000,840.834741,793.240322,2022-02-08,1,103281.269532
9,NaN,Indiana Pacers,Atlanta Hawks,0,0,0,0,0.351709,0.461587,0.710430,...,-0.138055,0.093491,0.000000,0.011629,0.000000,1191.874261,4886.684470,2022-02-08,0,102089.395271
